In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(16, input_dim=61, activation='linear', kernel_initializer='uniform', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
	# Compile model
	#optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 8
np.random.seed(seed)
# load dataset
dataset = pd.read_csv("modified-monthly-data-labelled.csv")
# split into input (X) and output (Y) variables
X = dataset.iloc[:,0:61]
#X = dataset[:, 2:]
Y = dataset.iloc[:,61]

In [8]:

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=20, epochs=100)
weight_constraint = [3]
dropout_rate = [0.2]

# define the grid search parameters
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000000 using {'dropout_rate': 0.2, 'weight_constraint': 3}
0.000000 (0.000000) with: {'dropout_rate': 0.2, 'weight_constraint': 3}
